In [50]:
import numpy as np # NumPy is the fundamental package for scientific computing

import pandas as pd # Pandas is an easy-to-use data structures and data analysis tools
pd.set_option('display.max_columns', None) # To display all columns

import matplotlib.pyplot as plt # Matplotlib is a python 2D plotting library
%matplotlib inline 
# A magic command that tells matplotlib to render figures as static images in the Notebook.

import seaborn as sns # Seaborn is a visualization library based on matplotlib (attractive statistical graphics).
sns.set_style('whitegrid') # One of the five seaborn themes
import warnings
warnings.filterwarnings('ignore') # To ignore some of seaborn warning msg

from scipy import stats

from sklearn import linear_model # Scikit learn library that implements generalized linear models
from sklearn import neighbors # provides functionality for unsupervised and supervised neighbors-based learning methods
from sklearn.metrics import mean_squared_error # Mean squared error regression loss
from sklearn import preprocessing # provides functions and classes to change raw feature vectors

from math import log

In [58]:
#data = pd.read_csv("./eggs.csv", parse_dates = ['date']) # load the data into a pandas dataframe
data = pd.read_csv("./eggs.csv") # load the data into a pandas dataframe
data.head(2) # Show the first 2 lines

,changeid,keyword,keywordid,time,hour,price,rank1rate,cpc,hourclick
0,89,导航下载,41204679123,2016-10-14 14:39:59,14,1.98,0.960784,0.9644,1.666667
1,88,导航下载,41204679123,2016-10-14 14:09:59,14,1.98,1.000000,0.9460,1.980198


In [59]:
data.drop(['changeid', 'keyword', 'time','hourclick'], axis = 1, inplace = True)

In [60]:
if False:
    categorial_cols = ['floors', 'view', 'condition', 'grade']

    for cc in categorial_cols:
        dummies = pd.get_dummies(data[cc], drop_first=False)
        dummies = dummies.add_prefix("{}#".format(cc))
        data.drop(cc, axis=1, inplace=True)
        data = data.join(dummies)

In [61]:
if True:
    dummies_zipcodes = pd.get_dummies(data['keywordid'], drop_first=False)
    dummies_zipcodes.reset_index(inplace=True)
    dummies_zipcodes = dummies_zipcodes.add_prefix("{}#".format('keywordid'))
    dummies_zipcodes = dummies_zipcodes[['keywordid#41204679123',
                                         'keywordid#41204679138',
                                         'keywordid#41204679141',
                                         'keywordid#51851122225',
                                         'keywordid#51683066718',
                                         'keywordid#52549511179',
                                         'keywordid#51683066700',
                                         'keywordid#52549510906',
                                        ]]
    data.drop('keywordid', axis=1, inplace=True)
    data = data.join(dummies_zipcodes)

data.dtypes

hour                       int64
price                    float64
rank1rate                float64
cpc                      float64
keywordid#41204679123      uint8
keywordid#41204679138      uint8
keywordid#41204679141      uint8
keywordid#51851122225      uint8
keywordid#51683066718      uint8
keywordid#52549511179      uint8
keywordid#51683066700      uint8
keywordid#52549510906      uint8
dtype: object

In [62]:
from sklearn.cross_validation import train_test_split
train_data, test_data = train_test_split(data, train_size = 0.8, random_state = 10)

In [63]:
# A function that take one input of the dataset and return the RMSE (of the test data), and the intercept and coefficient
def simple_linear_model(train, test, input_feature):
    regr = linear_model.LinearRegression() # Create a linear regression object
    regr.fit(train.as_matrix(columns = [input_feature]), train.as_matrix(columns = ['cpc'])) # Train the model
    RMSE = mean_squared_error(test.as_matrix(columns = ['cpc']), 
                              regr.predict(test.as_matrix(columns = [input_feature])))**0.5 # Calculate the RMSE on test data
    return RMSE, regr.intercept_[0], regr.coef_[0][0]

In [64]:
RMSE, w0, w1 = simple_linear_model(train_data, test_data, 'price')
print ('RMSE for sqft_living is: %s ' %RMSE)
print ('intercept is: %s' %w0)
print ('coefficient is: %s' %w1)

RMSE for sqft_living is: 0.583949613109 
intercept is: 0.613846424957
coefficient is: 0.322036357703


In [66]:
input_list = data.columns.values.tolist() # list of column name
input_list.remove('cpc')
simple_linear_result = pd.DataFrame(columns = ['feature', 'RMSE', 'intercept', 'coefficient'])

# loop that calculate the RMSE of the test data for each input 
for p in input_list:
    RMSE, w1, w0 = simple_linear_model(train_data, test_data, p)
    simple_linear_result = simple_linear_result.append({'feature':p, 'RMSE':RMSE, 'intercept':w0, 'coefficient': w1}
                                                       ,ignore_index=True)
simple_linear_result.sort_values('RMSE').head(10) # display the 10 best estimators

,feature,RMSE,intercept,coefficient
0,hour,0.560926,0.023797,0.702593
2,rank1rate,0.579475,0.300553,0.784571
1,price,0.583950,0.322036,0.613846
10,keywordid#52549510906,0.586009,-0.160840,1.040683
4,keywordid#41204679138,0.592905,0.098130,1.009906
7,keywordid#51683066718,0.593420,0.145405,0.995298
5,keywordid#41204679141,0.593729,-0.008799,1.017608
9,keywordid#51683066700,0.593865,0.019302,1.014194
3,keywordid#41204679123,0.593917,0.003829,1.016712
6,keywordid#51851122225,0.593926,-0.010622,1.018540


In [67]:
# A function that take multiple features as input and return the RMSE (of the test data), and the  intercept and coefficients
def multiple_regression_model(train, test, input_features):
    regr = linear_model.LinearRegression() # Create a linear regression object
    regr.fit(train.as_matrix(columns = input_features), train.as_matrix(columns = ['cpc'])) # Train the model
    RMSE = mean_squared_error(test.as_matrix(columns = ['cpc']), 
                              regr.predict(test.as_matrix(columns = input_features)))**0.5 # Calculate the RMSE on test data
    return RMSE, regr.intercept_[0], regr.coef_ 

In [68]:
print ('RMSE: %s, intercept: %s, coefficients: %s' %multiple_regression_model(train_data,test_data, ['price','rank1rate']))
print ('RMSE: %s, intercept: %s, coefficients: %s' %multiple_regression_model(train_data,test_data, ['price','hour']))
print ('RMSE: %s, intercept: %s, coefficients: %s' %multiple_regression_model(train_data,test_data, ['hour','rank1rate']))


RMSE: 0.5788456657, intercept: 0.548269476493, coefficients: [[ 0.2915453   0.13416812]]
RMSE: 0.552722139103, intercept: 0.256706807429, coefficients: [[ 0.33862712  0.02546134]]
RMSE: 0.554407928777, intercept: 0.538845431312, coefficients: [[ 0.02190629  0.24406471]]


In [71]:
train_data['hour_sq'] = train_data['hour'].apply(lambda x: x**2) # create a new column in train_data
test_data['hour_sq'] = test_data['hour'].apply(lambda x: x**2) # create a new column in test_data

train_data['price_2'] = train_data['price'].apply(lambda x: x**2)
test_data['price_2'] = test_data['price'].apply(lambda x: x**2)

train_data['rank1_2'] = train_data['rank1rate'].apply(lambda x: x**2)
test_data['rank1_2'] = test_data['rank1rate'].apply(lambda x: x**2) 

print ('RMSE: %s, intercept: %s, coefficients: %s' %multiple_regression_model(train_data, 
                                                                             test_data, ['price', 'price_2']))

RMSE: 0.586064155419, intercept: 0.329629663048, coefficients: [[ 0.80843972 -0.18295247]]


In [72]:
# we're first going to add more features into the dataset.

# sqft_living cubed
train_data['price_3'] = train_data['price'].apply(lambda x: x**3) 
test_data['price_3'] = test_data['price'].apply(lambda x: x**3) 

# bedrooms_squared: this feature will mostly affect houses with many bedrooms.
#train_data['rank1_0'] = train_data['rank1rate'].apply(lambda x: log(x)) 
#test_data['rank1_0'] = test_data['rank1rate'].apply(lambda x: log(x))

# bedrooms times bathrooms gives what's called an "interaction" feature. It is large when both of them are large.
#train_data['bed_bath_rooms'] = train_data['bedrooms']*train_data['bathrooms']
#test_data['bed_bath_rooms'] = test_data['bedrooms']*test_data['bathrooms']

# Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
#train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x: log(x))
#test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: log(x))

train_data.shape

(597, 16)

In [73]:
# split the train_data to include a validation set (train_data2 = 60%, validation_data = 20%, test_data = 20%)
train_data_2, validation_data = train_test_split(train_data, train_size = 0.75, random_state = 50)

In [74]:
# A function that take multiple features as input and return the RMSE (of the train and validation data)
def RMSE(train, validation, features, new_input):
    features_list = list(features)
    features_list.append(new_input)
    regr = linear_model.LinearRegression() # Create a linear regression object
    regr.fit(train.as_matrix(columns = features_list), train.as_matrix(columns = ['price'])) # Train the model
    RMSE_train = mean_squared_error(train.as_matrix(columns = ['price']), 
                              regr.predict(train.as_matrix(columns = features_list)))**0.5 # Calculate the RMSE on train data
    RMSE_validation = mean_squared_error(validation.as_matrix(columns = ['price']), 
                              regr.predict(validation.as_matrix(columns = features_list)))**0.5 # Calculate the RMSE on train data
    return RMSE_train, RMSE_validation 

In [75]:
input_list = train_data_2.columns.values.tolist() # list of column name
input_list.remove('cpc')

# list of features included in the regression model and the calculated train and validation errors (RMSE)
regression_greedy_algorithm = pd.DataFrame(columns = ['feature', 'train_error', 'validation_error'])  
i = 0
temp_list = []

# a while loop going through all the features in the dataframe
while i < len(train_data_2.columns)-1:
    
    # a temporary dataframe to select the best feature at each iteration
    temp = pd.DataFrame(columns = ['feature', 'train_error', 'validation_error'])
    
    # a for loop to test all the remaining features
    for p in input_list:
        RMSE_train, RMSE_validation = RMSE(train_data_2, validation_data, temp_list, p)
        temp = temp.append({'feature':p, 'train_error':RMSE_train, 'validation_error':RMSE_validation}, ignore_index=True)
        
    temp = temp.sort_values('train_error') # select the best feature using train error
    best = temp.iloc[0,0]
    temp_list.append(best)
    regression_greedy_algorithm = regression_greedy_algorithm.append({'feature': best, 
                                                  'train_error': temp.iloc[0,1], 'validation_error': temp.iloc[0,2]}, 
                                                 ignore_index=True) # add the feature to the dataframe
    input_list.remove(best) # remove the best feature from the list of available features
    i += 1
regression_greedy_algorithm

,feature,train_error,validation_error
0,price,1.356949e-16,1.257708e-16
1,keywordid#41204679123,0.000000e+00,0.000000e+00
2,keywordid#51683066700,0.000000e+00,0.000000e+00
3,keywordid#51683066718,9.415577e-17,9.365879e-17
4,price_2,1.564815e-16,1.556294e-16
5,keywordid#52549511179,9.735938e-17,9.784250e-17
6,rank1_2,1.844277e-16,1.745441e-16
7,rank1rate,1.373611e-16,1.375512e-16
8,keywordid#52549510906,1.185010e-16,1.232964e-16
9,keywordid#41204679141,3.482147e-16,3.514635e-16


In [78]:

greedy_algo_features_list = regression_greedy_algorithm['feature'].tolist()[:12] # select the first 30 features
test_error, _, _ = multiple_regression_model(train_data_2, test_data, greedy_algo_features_list)
print ('test error (RMSE) is: %s' %test_error)

test error (RMSE) is: 0.562738424538


In [77]:

input_feature = train_data.columns.values.tolist() # list of column name
input_feature.remove('cpc')

for i in [1,10]:
    ridge = linear_model.Ridge(alpha = i, normalize = True) # initialize the model
    ridge.fit(train_data.as_matrix(columns = input_feature), train_data.as_matrix(columns = ['cpc'])) # fit the train data
    print ('test error (RMSE) is: %s' %mean_squared_error(test_data.as_matrix(columns = ['cpc']), 
                              ridge.predict(test_data.as_matrix(columns = [input_feature])))**0.5) # predict price and test error

test error (RMSE) is: 0.383967842026
test error (RMSE) is: 0.38059134014


In [78]:
ridgeCV = linear_model.RidgeCV(alphas = np.linspace(1.0e-10,1,num = 100), normalize = True, store_cv_values = True) # initialize the model
ridgeCV.fit(train_data.as_matrix(columns = input_feature), train_data.as_matrix(columns = ['cpc'])) # fit the train data
print ('best alpha is: %s' %ridgeCV.alpha_) # get the best alpha
print ('test error (RMSE) is: %s' %mean_squared_error(test_data.as_matrix(columns = ['cpc']), 
                              ridgeCV.predict(test_data.as_matrix(columns = [input_feature])))**0.5) # predict price and test error

best alpha is: 0.0303030304
test error (RMSE) is: 0.387848881773


In [79]:
for i in [0.01,0.1,1,250,500,1000]:
    lasso = linear_model.Lasso(alpha = i, normalize = True) # initialize the model
    lasso.fit(train_data.as_matrix(columns = input_feature), train_data.as_matrix(columns = ['cpc'])) # fit the train data
    print (lasso.sparse_coef_.getnnz) # number of non zero weights
    print ('test error (RMSE) is: %s' %mean_squared_error(test_data.as_matrix(columns = ['cpc']), 
                              lasso.predict(test_data.as_matrix(columns = [input_feature])))**0.5) # predict price and test error

<bound method _cs_matrix.getnnz of <1x7 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>>
test error (RMSE) is: 0.379868689916
<bound method _cs_matrix.getnnz of <1x7 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>>
test error (RMSE) is: 0.380408408688
<bound method _cs_matrix.getnnz of <1x7 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>>
test error (RMSE) is: 0.380408408688
<bound method _cs_matrix.getnnz of <1x7 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>>
test error (RMSE) is: 0.380408408688
<bound method _cs_matrix.getnnz of <1x7 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>>
test error (RMSE) is: 0.380408408688
<bound method _cs_matrix.getnnz of <1x7 sparse matrix of type '<class 'numpy.float64'

In [26]:
lassoCV = linear_model.LassoCV(normalize = True) # initialize the model (alphas are set automatically)
lassoCV.fit(train_data.as_matrix(columns = input_feature), np.ravel(train_data.as_matrix(columns = ['price']))) # fit the train data
print ('best alpha is: %s' %lassoCV.alpha_) # get the best alpha
print ('number of non zero weigths is: %s' %np.count_nonzero(lassoCV.coef_)) # number of non zero weights
print ('test error (RMSE) is: %s' %mean_squared_error(test_data.as_matrix(columns = ['price']), 
                              lassoCV.predict(test_data.as_matrix(columns = [input_feature])))**0.5) # predict price and test error

best alpha is: 12.1151133713
number of non zero weigths is: 39
test error (RMSE) is: 171760.045248


In [27]:
# normalize the data
train_X = train_data.as_matrix(columns = input_feature)
scaler = preprocessing.StandardScaler().fit(train_X)
train_X_scaled = scaler.transform(train_X)
test_X = test_data.as_matrix(columns = [input_feature])
test_X_scaled = scaler.transform(test_X)

knn = neighbors.KNeighborsRegressor(n_neighbors=10, weights='distance') # initialize the model
knn.fit(train_X_scaled, train_data.as_matrix(columns = ['price'])) # fit the train data
print ('test error (RMSE) is: %s' %mean_squared_error(test_data.as_matrix(columns = ['price']), 
                              knn.predict(test_X_scaled))**0.5) # predict price and test error

test error (RMSE) is: 183023.744768
